# Stream-Static Joins

In this lesson, you'll join streaming heart rate data with the completed workouts table.

We'll be creating the table **`workout_bpm`** in our architectural diagram.

This pattern will take advantage of Delta Lake's ability to guarantee that the latest version of a table is returned each time it is queried.



<img src="https://files.training.databricks.com/images/ade/ADE_arch_workout_bpm.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Describe guarantees around versioning and matching for stream-static joins
- Leverage Spark SQL and PySpark to process stream-static joins

## Setup

**NOTE**: The setup script includes logic to define a **`user_lookup`** table required for the join below.

In [0]:
%run ../Includes/Classroom-Setup-4.5

Set up your streaming temp view. Note that we will only be streaming from **one** of our tables. The **`completed_workouts`** table is no longer streamable as it breaks the requirement of an ever-appending source for Structured Streaming. However, when performing a stream-static join with a Delta table, each batch will confirm that the newest version of the static Delta table is being used.

In [0]:
(spark.readStream
      .table("heart_rate_silver")
      .createOrReplaceTempView("TEMP_heart_rate_silver"))

## Perform Stream-Static Join to Align Workouts to Heart Rate Recordings

Below we'll configure our query to join our stream to our **`completed_workouts`** table.

Note that our heart rate recordings only have **`device_id`**, while our workouts use **`user_id`** as the unique identifier. We'll need to use our **`user_lookup`** table to match these values. Because all tables are Delta Lake tables, we're guaranteed to get the latest version of each table during each microbatch transaction.

Importantly, our devices occasionally send messages with negative recordings, which represent a potential error in the recorded values. We'll need to define predicate conditions to ensure that only positive recordings are processed.

In [0]:
%sql 
CREATE OR REPLACE TEMP VIEW TEMP_workout_bpm AS
  SELECT d.user_id, d.workout_id, d.session_id, time, heartrate
  FROM TEMP_heart_rate_silver c
  INNER JOIN (
    SELECT a.user_id, b.device_id, workout_id, session_id, start_time, end_time
    FROM completed_workouts a
    INNER JOIN user_lookup b
    ON a.user_id = b.user_id) d
  ON c.device_id = d.device_id AND time BETWEEN start_time AND end_time
  WHERE c.bpm_check = 'OK'

Note that the streaming portion of the join drives this join process. As currently implemented, this means that records from the **`heart_rate_silver`** table will only appear in our results table if a matching record has been written to the **`completed_workouts`** table prior to processing this query.

Stream-static joins are not stateful, meaning that we cannot configure our query to wait for records to appear in the right side of the join prior to calculating the results. When leveraging stream-static joins, make sure to be aware of potential limitations for unmatched records. (Note that a separate batch job could be configured to find and insert records that were missed during incremental execution).

### Write Stream in Append Mode

Below, we'll use our streaming temp view from above to insert new values into our **`workout_bpm`** table.

In [0]:
def process_workout_bpm():
    query = (spark.table("TEMP_workout_bpm")
                  .writeStream
                  .format("delta")
                  .outputMode("append")
                  .option("checkpointLocation", f"{DA.paths.checkpoints}/workout_bpm")
                  .trigger(availableNow=True)
                  .table("workout_bpm"))
    
    query.awaitTermination()
    
process_workout_bpm()

Explore this results table below.

In [0]:
%sql

SELECT COUNT(*)
FROM workout_bpm

In [0]:
%sql

SELECT * FROM workout_bpm

If desired, process another batch through all tables and update these results.

In [0]:
DA.daily_stream.load()          # Load one new day for DA.paths.source_daily
DA.process_bronze()             # Process through the bronze table
DA.process_heart_rate_silver()  # Process the heart_rate_silver table
DA.process_workouts_silver()    # Process the workouts_silver table
DA.process_completed_workouts() # Process the completed_workouts table

process_workout_bpm()

In [0]:
%sql

SELECT COUNT(*)
FROM workout_bpm

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()